In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
#from sklearn.linear_model import LinearRegression 
from sklearn import metrics 
#import matplotlib.pyplot as plt 
import numpy as np 
#import seaborn as sns
import mlflow
from mlflow import sklearn
from sklearn import preprocessing
from sklearn import tree

In [2]:
data = pd.read_csv('winequality-red.csv', sep=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
data.min()

fixed acidity           4.60000
volatile acidity        0.12000
citric acid             0.00000
residual sugar          0.90000
chlorides               0.01200
free sulfur dioxide     1.00000
total sulfur dioxide    6.00000
density                 0.99007
pH                      2.74000
sulphates               0.33000
alcohol                 8.40000
quality                 3.00000
dtype: float64

In [4]:
data.max()

fixed acidity            15.90000
volatile acidity          1.58000
citric acid               1.00000
residual sugar           15.50000
chlorides                 0.61100
free sulfur dioxide      72.00000
total sulfur dioxide    289.00000
density                   1.00369
pH                        4.01000
sulphates                 2.00000
alcohol                  14.90000
quality                   8.00000
dtype: float64

In [5]:
# there are no categorical variables. each feature is a number. Regression problem. 
# Given the set of values for features, we have to predict the quality of wine. finding correlation of each feature with our target variable - quality
#correlations = df.corr()['quality'].drop('quality')
#print(correlations)

y = data.quality
X = data.drop('quality', axis=1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)
X_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
434,10.4,0.41,0.55,3.2,0.076,22.0,54.0,0.99960,3.15,0.89,9.9
623,7.9,0.51,0.25,2.9,0.077,21.0,45.0,0.99740,3.49,0.96,12.1
1010,8.9,0.28,0.45,1.7,0.067,7.0,12.0,0.99354,3.25,0.55,12.3
1355,6.1,0.32,0.25,1.8,0.086,5.0,32.0,0.99464,3.36,0.44,10.1
900,8.9,0.31,0.36,2.6,0.056,10.0,39.0,0.99562,3.40,0.69,11.8


In [7]:
X_train_scaled = preprocessing.scale(X_train)
X_train_scaled

array([[ 1.19515058, -0.63699091,  1.42947102, ..., -1.02699388,
         1.31708452, -0.49841813],
       [-0.24915721, -0.08349971, -0.11791946, ...,  1.16770148,
         1.71974315,  1.55981967],
       [ 0.32856591, -1.35652947,  0.9136742 , ..., -0.38149524,
        -0.63868593,  1.7469322 ],
       ...,
       [ 0.7907444 , -1.13513299,  1.37789134, ..., -0.18784565,
         0.9144259 , -0.77908692],
       [ 0.61742746,  0.8851099 , -0.06633978, ..., -0.18784565,
        -0.63868593, -0.96619945],
       [ 0.7907444 , -1.19048211,  1.01683356, ..., -0.25239552,
        -0.00593666, -0.40486187]])

In [8]:
clf=tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [9]:
confidence = clf.score(X_test, y_test)
print("\nThe confidence score:\n")
print(confidence)


The confidence score:

0.628125


In [10]:
y_pred = clf.predict(X_test)

In [11]:

#converting the numpy array to list
x=np.array(y_pred).tolist()

#printing first 5 predictions
print("\nThe prediction:\n")
for i in range(0,5):
    print( x[i])
    
#printing first five expectations
print("\nThe expectation:\n")
print( y_test.head())


The prediction:

6
8
5
6
6

The expectation:

610     5
989     6
1496    5
530     6
54      6
Name: quality, dtype: int64


In [12]:
mlflow.set_experiment('WineQuality')
with mlflow.start_run():
    
    mlflow.log_metric("rmse", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    mlflow.log_metric("r2", metrics.mean_squared_error(y_test, y_pred))
    mlflow.log_metric("mae", metrics.mean_absolute_error(y_test, y_pred))
    
    conda = {
        'name': 'mlflow-env',
        'channels': ['defaults'],
        'dependencies': [
            'python=3.7.0',
            'scikit-learn=0.19.2'
        ]
    }

    mlflow.sklearn.log_model(clf, 'model', serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_CLOUDPICKLE, conda_env = conda)
    print(mlflow.get_artifact_uri())

file:/mlflow/1/a9a2714a49b6445baafb15a797859d4b/artifacts
